In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, roc_curve, auc

# week4 授業課題 信用情報の学習
# 【問題1】コンペティション内容の確認
### コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。  
### 何を学習し、何を予測するのか どのようなファイルを作りKaggleに提出するか 提出されたものはどういった指標値で評価されるのか

122個ある特徴量からその人が返済する確率を予測する
IDとTARGETがカラムとして入っているcsvファイルを提出する
TARGETにはその人が返済する確率が入っている


提出された課題はAUCで評価される

# 【問題2】学習と検証
データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。
検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。

In [2]:
os.getcwd()

'C:\\Users\\ykenk\\dive\\diveintocode-ml\\week4'

In [3]:
test_data_raw = pd.read_csv("/Users\\ykenk\\dive\\diveintocode-ml\\week4/application_test.csv")
train_data_raw=pd.read_csv("/Users\\ykenk\\dive\\diveintocode-ml\\week4/application_train.csv")

In [4]:
train_data = train_data_raw.copy(deep=True)
test_data = test_data_raw.copy(deep=True)

In [5]:
#欠損値がある列を削除
droped_train = train_data.dropna(axis=1)
droped_test = test_data.dropna(axis=1)

#train, testのデータで共通する列を抜きだす
drop_column_intersection = droped_train.columns & droped_test.columns

droped_train_inter = train_data[drop_column_intersection]
droped_train_inter.insert(1, "TARGET", train_data.loc[:, "TARGET"])
droped_test_inter = test_data[drop_column_intersection]

#機械学習させる上で文字列を使えないので、objectタイプを削除する
on_train = droped_train_inter.select_dtypes(include="number")
on_test = droped_test_inter.select_dtypes(include="number")

#機械学習用のtrain_test_splitのために説明変数と目的変数を分ける
X = on_train.drop(columns="TARGET")
y = on_train["TARGET"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [7]:
RFC = RandomForestClassifier(n_estimators=10)
RFC.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# 【問題3】テストデータに対する推定
テストデータ（application_test.csv）に対して推定を行い、Kaggleに提出を行ってください。

正しく提出が行えていれば、精度は低くても構いません。

In [8]:

y_pred_1 = RFC.predict(X_test)
fpr_1, tpr_1, thresholds_1 = roc_curve(y_test, y_pred_1)
auc_1 = auc(fpr_1, tpr_1)
print("accuracy：", accuracy_score(y_test, y_pred_1))
print("recall:", recall_score(y_test, y_pred_1))
print("precision:", precision_score(y_test, y_pred_1))
print("f1:",f1_score(y_test, y_pred_1))
print("matrix:\n", confusion_matrix(y_test, y_pred_1))
print("-"*20)
print("auc:",auc_1)

accuracy： 0.920003121829392
recall: 0.0016417665407978984
precision: 0.12658227848101267
f1: 0.0032414910858995136
matrix:
 [[70718    69]
 [ 6081    10]]
--------------------
auc: 0.5003335056445637


### kaggle提出　　public_score:0.49024¶

# 【問題4】特徴量エンジニアリング
精度を上げるために以下のような観点で 特徴量エンジニアリング（Feature Engineering） を行ってください。
どの特徴量を使うか
どう前処理をするか
何をした時に検証データに対する評価指標がどのようになったかをまとめてください。最低5パターンの学習・検証を行ってください。


精度が高かったものに関してはテストデータに対しても推定を行い、Kaggleに提出を行ってください。

## パターン１：RandomForestの重要な特徴量でソートし、高い方から５つ選んで学習する

In [9]:
features = X_train.columns
importances = RFC.feature_importances_

print("重要度が高い特徴量順:")
print(sorted(zip(map(lambda x: round(x, 2), RFC.feature_importances_), features), 
             reverse=True))

重要度が高い特徴量順:
[(0.12, 'DAYS_REGISTRATION'), (0.12, 'DAYS_ID_PUBLISH'), (0.12, 'DAYS_BIRTH'), (0.11, 'SK_ID_CURR'), (0.1, 'DAYS_EMPLOYED'), (0.1, 'AMT_CREDIT'), (0.08, 'REGION_POPULATION_RELATIVE'), (0.08, 'AMT_INCOME_TOTAL'), (0.07, 'HOUR_APPR_PROCESS_START'), (0.02, 'CNT_CHILDREN'), (0.01, 'REG_CITY_NOT_WORK_CITY'), (0.01, 'REGION_RATING_CLIENT_W_CITY'), (0.01, 'REGION_RATING_CLIENT'), (0.01, 'FLAG_WORK_PHONE'), (0.01, 'FLAG_PHONE'), (0.01, 'FLAG_EMAIL'), (0.01, 'FLAG_DOCUMENT_3'), (0.0, 'REG_REGION_NOT_WORK_REGION'), (0.0, 'REG_REGION_NOT_LIVE_REGION'), (0.0, 'REG_CITY_NOT_LIVE_CITY'), (0.0, 'LIVE_REGION_NOT_WORK_REGION'), (0.0, 'LIVE_CITY_NOT_WORK_CITY'), (0.0, 'FLAG_MOBIL'), (0.0, 'FLAG_EMP_PHONE'), (0.0, 'FLAG_DOCUMENT_9'), (0.0, 'FLAG_DOCUMENT_8'), (0.0, 'FLAG_DOCUMENT_7'), (0.0, 'FLAG_DOCUMENT_6'), (0.0, 'FLAG_DOCUMENT_5'), (0.0, 'FLAG_DOCUMENT_4'), (0.0, 'FLAG_DOCUMENT_21'), (0.0, 'FLAG_DOCUMENT_20'), (0.0, 'FLAG_DOCUMENT_2'), (0.0, 'FLAG_DOCUMENT_19'), (0.0, 'FLAG_DOCUMENT_18'),

### 使う特徴量を5つにする  
'DAYS_REGISTRATION'  
'DAYS_BIRTH'  
'DAYS_ID_PUBLISH'  
'DAYS_EMPLOYED'  
'AMT_CREDIT'  

In [10]:
importance_feature = ["DAYS_REGISTRATION", "DAYS_BIRTH", "DAYS_ID_PUBLISH", "DAYS_EMPLOYED", "AMT_CREDIT"]
train_data_pattern1 = on_train[importance_feature]
test_data_pattern1 = on_test[importance_feature]
TARGET = on_train["TARGET"]

In [11]:
X_pattern1 = train_data_pattern1
y_pattern1 = TARGET

In [12]:
X_train_pattern1, X_test_pattern1, y_train_pattern1, y_test_pattern1 = train_test_split(X_pattern1, y_pattern1, test_size=0.25,
                                                                                       random_state=42)

In [13]:
RFC_pattern1 = RandomForestClassifier(n_estimators=100)

In [16]:
RFC_pattern1.fit(X_train_pattern1, y_train_pattern1)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [17]:
y_pred_pattern1 = RFC_pattern1.predict(X_test_pattern1)

In [18]:
fpr_pattern1, tpr_pattern1, thresholds_pattern1 = roc_curve(y_test_pattern1, y_pred_pattern1)
auc_pattern1 = auc(fpr_pattern1, tpr_pattern1)
print("accuracy：", accuracy_score(y_test, y_pred_pattern1))
print("recall:", recall_score(y_test, y_pred_pattern1))
print("precision:", precision_score(y_test, y_pred_pattern1))
print("f1:",f1_score(y_test, y_pred_pattern1))
print("matrix:\n", confusion_matrix(y_test, y_pred_pattern1))
print("-"*20)
print("auc:",auc_pattern1)

accuracy： 0.92061448008533
recall: 0.0
precision: 0.0
f1: 0.0
matrix:
 [[70775    12]
 [ 6091     0]]
--------------------
auc: 0.5002664620967768


## ３つ抜き出して　学習させる¶

In [19]:

pattern2_selected_columns = ["AMT_INCOME_TOTAL", "NAME_HOUSING_TYPE", "NAME_EDUCATION_TYPE"]

In [20]:
train_data_pattern2 = train_data[pattern2_selected_columns].copy(deep=True)

In [21]:
train_data_pattern2 = train_data[pattern2_selected_columns].copy(deep=True)

In [22]:
X_train_pattern2, X_test_pattern2, y_train_pattern2, y_test_pattern2 = train_test_split(train_data_pattern2, TARGET, test_size=0.25,
                                                                                       random_state=42)

In [23]:
RFC_pattern2 = RandomForestClassifier(n_estimators=100)

In [24]:
RFC_pattern2.fit(X_train_pattern2, y_train_pattern2)

ValueError: could not convert string to float: 'House / apartment'

## ここまでしかやっていません。